In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%precision 4

'%.4f'

In [157]:
userDframe = pd.read_csv('data.csv')
cleanedData = cleanAndNormalizeData(userDframe)

iniData = randomRating(cleanedData)
getOptimalTheta(iniData)


[[ 0.4504]
 [-0.0024]
 [ 0.0029]] 10134055616513
[[ 0.4218]
 [ 0.0399]
 [-0.1263]] 10128184674056
[[ 4.4152e-01]
 [-7.4543e-03]
 [-8.8218e-05]] 10134011535754
[[0.4292]
 [0.0274]
 [0.0101]] 10134016049164
[[ 0.4156]
 [-0.0271]
 [ 0.0049]] 10134012147603
[[ 0.2261]
 [ 0.2261]
 [-0.0056]] 10128110003733
[[ 0.4341]
 [ 0.0029]
 [-0.0451]] 10160362424221
[[0.446 ]
 [0.004 ]
 [0.0005]] 10134040303649
[[0.3446]
 [0.1097]
 [0.0179]] 10134051513507
[[ 0.452 ]
 [-0.0043]
 [-0.002 ]] 10128184954279
[[ 0.2229]
 [ 0.2229]
 [-0.0726]] 10153747215666
[[ 0.2268]
 [-0.2268]
 [ 0.011 ]] 10134071009718
[[ 0.448 ]
 [-0.0014]
 [-0.0246]] 10128110002624
[[0.4452]
 [0.0179]
 [0.0121]] 10134023845956
[[ 0.2148]
 [-0.2148]
 [ 0.1069]] 10153719210054
[[ 0.4427]
 [-0.0028]
 [ 0.0012]] 10153747215688
[[ 0.4507]
 [-0.001 ]
 [-0.0018]] 10134015329369
[[ 0.3925]
 [-0.0603]
 [ 0.0054]] 10128188699866
[[ 4.5042e-01]
 [-3.7655e-03]
 [-2.2349e-04]] 10134011588060
[[ 0.2139]
 [-0.2139]
 [-0.0744]] 10134018256604
[[0.4533

In [158]:
def getOptimalTheta(iniData):
    userList = list(set(iniData['user_id']))
    iniData['bias_unit'] = 1
    #iniData.to_csv('out.csv', encoding='utf-8', index=False)
    
    iniTheta = [[0],[0],[0]]
    alpha = 0.01
    num_iters = 1500
    optimal =  []
    for user in userList:
        tempItems = iniData.loc[iniData['user_id'] == user]
        userItems = tempItems[['bias_unit','swipe_direction','z_score','rating']]
        featureVec = userItems.iloc[:,[0,1,2]].values
        resultVec = userItems.iloc[:,[3]].values
        optimalTheta = gradientDescent(featureVec,resultVec,iniTheta,alpha,num_iters)
        #cost=computeCost(X,y,optimalTheta)
        #iniData.to_csv('out.csv', encoding='utf-8', index=False)
        print(optimalTheta, user)

In [159]:
#Cleaning data
#Dropping rows where impression_time<=0; because without seeing the user can't take any swipe action
#Dropping rows where impression_time >= 90000ms; this will be done because almost 99% values are in this range(<100000ms).

def cleanAndNormalizeData(userDataframe):
    indexNames = userDataframe[(userDataframe['impression_time'] <= 0) | (userDataframe['impression_time'] > 100000)].index
    modifiedData= userDataframe.drop(indexNames, axis=0)
    modifiedData['swipe_direction'].replace('SWIPED_OUT',-1,inplace = True)
    modifiedData['swipe_direction'].replace('SWIPED_IN',1,inplace = True)
    
    modifiedNormalizedData = getZScore(modifiedData)
    return modifiedNormalizedData

In [160]:
#Using Z-score normaliztion to normalize the impression_time column

def getZScore(modifiedData):
    z_score = (modifiedData['impression_time'] - modifiedData['impression_time'].mean())/modifiedData['impression_time'].std()
    modifiedData['z_score'] = z_score
    return modifiedData

In [164]:
def randomRating(cleanedData):
    cleanedData['rating'] = np.random.randint(4000,5000,cleanedData.shape[0])
    cleanedData['rating'] = cleanedData['rating']/10000  
        
    return cleanedData

In [162]:
def computeCost(featureVec, resultVec, theta):
    m = resultVec.size
    J = 0
    
    hypothesis = featureVec.dot(theta)
    
    J = 1/(2*m)*np.sum(np.square(hypothesis-resultVec))
    
    return(J)

In [165]:
def gradientDescent(featureVec, resultVec, theta, alpha, num_iters):
    m = resultVec.size
    J_history = np.zeros(num_iters)
    
    for iter in np.arange(num_iters):
        hypothesis = featureVec.dot(theta)
        theta = theta - alpha*(1/m)*(featureVec.T.dot(hypothesis-resultVec))
    return(theta)